In [59]:
import requests, os
from bs4 import BeautifulSoup
from src.downloader import download

def get_html(url):
    """Get raw HTML from a URL."""
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }
    req = requests.get(url, headers=headers)
    return req.text
    

def get_image(html):
    """Scrape share image."""
    image = []
    if html.find("meta", property="image"):
        image = html.find("meta", property="image").get('content')
    elif html.find("meta", property="og:image"):
        image = html.find("meta", property="og:image").get('content')
    elif html.find("meta", property="twitter:image"):
        image = html.find("meta", property="twitter:image").get('content')
    elif html.find("img", src=True):
        for img in html.find_all("img"):
            image.append(img.get('src'))
    return image


def check_word(word):
    def check(url):
        return (word in url)
    return check

In [14]:
keyword = "gadget electronique"
page = "2"
url = "https://www.amazon.fr/s?k={}&page={}&__mk_fr_FR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=sr_pg_{}".format(keyword, page, page)

soup = BeautifulSoup(get_html(url))
# print(soup.prettify())
for image_url in list(filter(check_word("_AC_UL320_"), get_image(soup))):
    download(image_url, "imgs")

SSLError: HTTPSConnectionPool(host='m.media-amazon.com', port=443): Max retries exceeded with url: /images/I/61YC5v2tk7L._AC_UL320_.jpg (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1076)')))

In [143]:
import os, requests, time, shutil, glob, bz2, imgkit, re
import numpy as np
from PIL import Image
from multiprocessing.dummy import Pool
from src.azurekey import *
from src.downloader import download

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
}

def search(search_term, azureKey, offset=0):
    # print('searching using bing: "'+search_term+'"')
    search_url = "https://api.bing.microsoft.com/v7.0/search"
    subscription_key = azureKey
    assert subscription_key
    headers = {"Ocp-Apim-Subscription-Key" : subscription_key}
    params  = {"q": search_term, "responseFilter": "Webpages","count":50, "offset":offset, "safeSearch":"Strict"}
    response = requests.get(search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()
    return [v["url"] for v in search_results["webPages"]["value"]]

def clean(html):
    html_temp = re.sub('href=".*?"', 'href=""', html)
    html_temp = re.sub('src=".*?"', 'src=""', html_temp)
    html_temp = re.sub('<script.*?/script>', '', html_temp)
    return html_temp

def save_html2jpg(url, dir_name):
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
    req = requests.get(url, headers=headers)
    imgkit.from_string(clean(req.text), os.path.join(dir_name, str(hash(url))+".jpg"))


# def download(links_in, dir_name, n=None, log=set()):
#     links = links_in[:n]
#     # print('search results',len(links)
#     if not os.path.exists(dir_name):
#         os.mkdir(dir_name)

#     def fetch(url, log=log):
#         r=requests.get(url[0], stream=True)
#         r.raw.decode_content = True # handle spurious Content-Encoding
#         img = Image.open(r.raw)
#         bits = str(imagehash.average_hash(img))
#         if bits not in log:
#             log |= set([bits])
#         if len(log) % 100 == 0:
#             print("{} images saved.".format(len(log)))
#         img_file = dir_name+'/commercial_product_'+bits+".jpg"
#         img.save(img_file)
        

#     modLinks=[[links[i],i+1] for i in range(len(links))]
#     Pool(10).map(fetch, modLinks)

#     return log

In [16]:
search_results = search("electronics", azureKey, offset=0)

In [121]:
url = "https://www.google.com/"
req = requests.get(url, headers=headers)
# imgkit.from_url(url, os.path.join("webpages", str(hash(url))+".jpg"))

In [122]:
def repair_link(html, url):
#     html_temp = re.sub('href="/', 'href="{}'.format(url), html)
#     html_temp = re.sub('src="/', 'src="{}'.format(url), html_temp)
#     html_temp = re.sub("url\(/", "url({}".format(url), html_temp)
#     html_temp = re.sub('poster="/', 'poster="{}'.format(url), html_temp)
#     html_temp = re.sub('srcset="/', 'srcset="{}'.format(url), html_temp)
    html_temp = re.sub('"/([^/"][^<"]*?)"', r'"{}\1"'.format(url), html)
    html_temp = re.sub("'/([^/'][^<']*?)'", r'"{}\1"'.format(url), html_temp)
    html_temp = re.sub("url\(/", "url({}".format(url), html_temp)
    return html_temp

In [123]:
with open("webpages/test.html", "w") as f:
#     f.write(req.text)
    f.write(repair_link(req.text, url))

In [142]:
re.findall('[^ \'()"]*?\.png', req.text)

['/images/branding/googleg/1x/googleg_standard_color_128dp.png',
 '//ssl.gstatic.com/gb/images/i1_1967ca6a.png',
 '//ssl.gstatic.com/gb/images/i2_2ec824b0.png',
 '/images/experiments/wavy-underline.png',
 '/images/searchbox/desktop_searchbox_sprites302_hr.png',
 '/images/searchbox/desktop_searchbox_sprites302_hr.png',
 '/images/searchbox/desktop_searchbox_sprites302_hr.png',
 '/images/branding/googlelogo/2x/googlelogo_color_272x92dp.png',
 '/images/branding/googlelogo/1x/googlelogo_color_272x92dp.png',
 '/images/branding/googlelogo/2x/googlelogo_color_272x92dp.png',
 'https://ssl.gstatic.com/gb/images/p2_8e348e57.png',
 '//ssl.gstatic.com/gb/images/i1_1967ca6a.png',
 '//ssl.gstatic.com/gb/images/i2_2ec824b0.png',
 '/images/nav_logo299.png']

In [188]:
def download(url, temp_dir, webpage_dir):
    save_path = webpage_dir + "/" + temp_dir
    r = requests.get(url, allow_redirects=True, headers=headers)
    file_name = os.path.split(url)[-1]
    print(file_name)
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    save_file_path = save_path + "/" + file_name
    open(save_file_path, 'wb').write(r.content)
    return temp_dir + "/" + file_name

def download_all(link_list, url, log, temp_dir="temp", webpage_dir="webpages"):
    for link in link_list:
        if link in log:
            continue
        if link.startswith("http"):
            log[link] = download(link, temp_dir, webpage_dir)
        elif link.startswith("//"):
            try:
                log[link] = download("https:" + link, temp_dir, webpage_dir)
            except:
                pass
        elif link.startswith("/"):
            link_temp = url+link[1:]
            log[link] = download(link_temp, temp_dir, webpage_dir)
    return log

def replace_all(html, log):
    html_temp = html
    for e, v in log.items():
        html_temp = re.sub(e, v, html_temp)
    return html_temp

In [189]:
url = "https://www.accenture.com/"
req = requests.get(url, headers=headers)

In [190]:
log = {}
log = download_all(re.findall('[^ \'()"]*?\.png', req.text), url, log, temp_dir="temp")
log = download_all(re.findall('[^ \'()"]*?\.jpg', req.text), url, log, temp_dir="temp")
log = download_all(re.findall('[^ \'()"]*?\.js', req.text), url, log, temp_dir="temp")
log = download_all(re.findall('[^ \'()"]*?\.css', req.text), url, log, temp_dir="temp")
log = download_all(re.findall('[^ \'()"]*?\.json', req.text), url, log, temp_dir="temp")

Accenture-Living-Business-Twitter.png
Accenture-Homepage-Image-400x400.jpg
Accenture-Home-Social-Image.jpg
France.jpg
Accenture-we-act-for-impact-1920x1080.jpg
70a2765e-e451-4bb1-83dd-6e13d136a97f.js
otSDKStub.js
wDq4Hx70.min.js
launch-EN664f8f34ad5946f8a0f7914005f717cf.min.js
ip.js
embed.js
ScrollMagic.min.js
debug.addIndicators.min.js
gsap.min.js
TweenLite.min.js
TweenMax.min.js
slick.js
ionicons.js
quick-fix-js.js
quick-fix-css.css
slick.css
slick-theme.css
ip.json


In [191]:
with open("webpages/test.html", "w") as f:
    f.write(replace_all(req.text, log))

In [22]:
dir_name = "webpages"
url = "https://www.twitter.com/"
search_url = "https://api.apiflash.com/v1/urltoimage?access_key=37bc0ab0376f4a33b4bac2e7a47776c4&height=1920&url={}".format(url)
r=requests.get(search_url, stream=True)
r.raw.decode_content = True # handle spurious Content-Encoding
img = Image.open(r.raw)
img_file = os.path.join(dir_name, str(hash(url))+".jpg")
img.save(img_file)